In [1]:
import numpy as np
import glob
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy import ndimage
from skimage.measure import moments
from skimage.feature import hog
import matplotlib
from scipy.interpolate import interp2d

# open first image to get the image dimensions
im1 = np.array(Image.open('sargentii_17/batch3_tomo_areaij_sample17.0503.tif'),dtype=np.double)

clustered_img1 = KMeans(2).fit_predict(im1.reshape([np.prod(np.shape(im1)),1]))            
clustered_img1[clustered_img1 == clustered_img1[0]] = 2e0
clustered_img1[clustered_img1 != clustered_img1[0]] = 3e0
clustered_img1 = clustered_img1 - 2e0
clustered_img1 = clustered_img1.reshape(np.shape(im1))
clustered_img1 = np.array(clustered_img1,dtype=np.double)

m1 = moments(clustered_img1)
cr1 = m1[0, 1] / m1[0, 0]
cc1 = m1[1, 0] / m1[0, 0]


feature_vector, hog_image = hog(im1,orientations=9,pixels_per_cell=(15,15),cells_per_block=(1,1),visualise=True,normalise=True)

print np.shape(feature_vector)
print np.shape(hog_image)


plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.axis('equal')
plt.ylim([0,np.shape(im1)[0]])
plt.xlim([0,np.shape(im1)[1]])
plt.contourf(im1,cmap='Greys_r')
#plt.scatter(cr1,cc1,color='r')

plt.subplot(1, 2, 2)
plt.ylim([0,np.shape(hog_image)[0]])
plt.xlim([0,np.shape(hog_image)[1]])
plt.imshow(hog_image)
plt.savefig('imgs/hog_image.png',dpi=150)

plt.show()


(16236,)
(667, 620)


In [13]:
import numpy as np
from PIL import Image
import os
import pickle
from scipy.misc import imresize
import pylab as pl

# label a set of tiles: labels can be background, leaf, or shell
# save the training data

# number of tiles/class to find
# all tiles will be rotated by 90, 180, and 270 degrees to increase sample size and ensure rotation invariance
n = 100

# pixel size of the tiles
# I found that I need 300x300 tiles to surely identify whether there is a whale on the image or only water
size = 30

# arrays to save stuff
tiles = np.zeros([size,size,3*n])
labels = np.zeros(3*n)
i = 0
ii = 0
iii = 0
count = 0
while count < 3*n:
    # get random pixel from the image
    x = np.random.randint(np.shape(im1)[0]-size)
    y = np.random.randint(np.shape(im1)[1]-size)

    plt.contourf(im1.T,cmap='Greys_r')
    square = plt.Rectangle((x,y),size,size,color='r',fill=False)
    plt.gca().add_patch(square)
    plt.contourf(im1[x:x+size,y:y+size].T)
    coord = pl.ginput(1,timeout=0)
    pl.close()

    # click on the left if it's background
    if coord[0][0] < np.shape(im1)[0]/3 and i < n:
        tiles[:,:,count] = im1[x:x+size,y:y+size]
        labels[count] = 0
        i = i + 1
        count = count + 1
    # click on the right if it is leaf:
    elif coord[0][0] > 2*np.shape(im1)[0]/3 and ii < n:
        tiles[:,:,count] = im1[x:x+size,y:y+size]
        labels[count] = 1  
        ii = ii + 1 
        count = count + 1
    # click to the middle if it is shell
    elif coord[0][0] > np.shape(im1)[0]/3 and coord[0][0] < 2*np.shape(im1)[0]/3 and iii < n:
        tiles[:,:,count] = im1[x:x+size,y:y+size]
        labels[count] = 2
        iii = iii + 1 
        count = count + 1

file_name = 'training_data_'+str(n)+'_'+str(size)+'x'+str(size)+'.dat'

# rotate the tiles and pickle dump the arrays
f = open(file_name,'w')
new_tiles = np.append(tiles,np.rot90(tiles,k=1),axis=2)
new_labels = np.append(labels,labels)

print np.shape(tiles),np.shape(labels)
print np.shape(new_tiles),np.shape(new_labels)

new_tiles = np.append(new_tiles,np.rot90(tiles,k=2),axis=2)
new_labels = np.append(new_labels,labels)

print np.shape(new_tiles),np.shape(new_labels)

new_tiles = np.append(new_tiles,np.rot90(tiles,k=3),axis=2)
new_labels = np.append(new_labels,labels)

print np.shape(new_tiles),np.shape(new_labels)

print new_labels

pickle.dump([new_tiles,new_labels],f)
f.close()    

print 'finished'

(30, 30, 300) (300,)
(30, 30, 600) (600,)
(30, 30, 900) (900,)
(30, 30, 1200) (1200,)
[ 0.  1.  2. ...,  1.  1.  1.]
finished


In [28]:
#do PCA
import numpy as np
import pylab as plt
'''
Performs the Principal Coponent analysis of the Matrix X
Matrix must be n * m dimensions
where n is # features
m is # examples
'''

def PCA(X, varRetained = [0.95],filename = 'PCA_data.dat'):

    # Compute Covariance Matrix Sigma
    (n, m) = X.shape

    Sigma = 1.0 / float(m) * np.dot(X, np.transpose(X))
    # Compute eigenvectors and eigenvalues of Sigma
    U, s, V = np.linalg.svd(Sigma)

    # compute the value k: number of minumum features that 
    # retains the given variance
    s_tot = np.sum(s)
        
    var_i = np.array([np.sum(s[: i + 1]) / s_tot * 100.0 for i in range(n)])
    
    k = np.zeros(len(varRetained))
    for i in range(len(k)):
        k[i] = len(var_i[var_i < (varRetained[i] * 100e0)])

        print '%.2f %% variance retained in %d dimensions' % (var_i[k[i]], k[i])

        # compute the reduced dimensional features 
        U_reduced = U[:, : k[i]]
        Z = np.dot(np.transpose(U_reduced),X)

        # pickle dump the results
        f = open(filename+str(int(varRetained[i]*100e0))+'.dat','w')
        pickle.dump([Z, U_reduced, k[i]],f)
        f.close() 

    return 



var_ret = [0.80, 0.85, 0.90, 0.95, 0.99] 

# load the training data and divide it to training (60%), test (20%), and cross validation (20%) sets
f = open('training_data_100_30x30.dat','r')
[tiles,labels] = pickle.load(f)
f.close()

# standardize tiles
tiles_standard = np.zeros(np.shape(tiles))
for i in range(1200):
    tiles_standard[:,:,i] = (tiles[:,:,i] - np.mean(im1) ) / np.std(im1)

# reshape tiles
reshape_tiles = np.reshape(tiles_standard,[30*30,1200])

filename = 'PCA_data_30x30_var0'

# do PCA and save the results
PCA(reshape_tiles,varRetained = var_ret,filename = filename)

print 'finished'



/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:30: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:33: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


81.43 % variance retained in 6 dimensions
85.85 % variance retained in 9 dimensions
90.17 % variance retained in 14 dimensions
95.12 % variance retained in 28 dimensions
99.01 % variance retained in 99 dimensions
finished


In [29]:
# obtain the dataset and run the SVM
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams['figure.figsize'] = (9.0, 6.0)
from sklearn.svm import SVC
import pickle
from scipy.misc import imresize

size = 30
# number of simulations 
n_sim = 100

file_name = 'training_data_100_'+str(size)+'x'+str(size)+'.dat'
f = open(file_name,'r')
[X,Y] = pickle.load(f)
f.close()  


f = open('PCA_data_30x30_var099.dat','r')
[Z, U_reduced, k] = pickle.load(f)
f.close()


# range of C and gamma values
# the range used to be wider: -8e0,3e0 with num = 12
# experience showed that the best C is within this reduced range usually
C = 10e0**(np.linspace(-4e0,4e0,9))
gamma = 10e0**(np.linspace(-4e0,4e0,9))

train_score = np.zeros([n_sim,len(C),len(gamma)])
test_score = np.zeros([n_sim,len(C),len(gamma)])
CV_score = np.zeros([n_sim,len(C),len(gamma)])

for i in range(n_sim):
    print '   ',i
    # shuffle and split data
    indx = np.arange(np.shape(Z.T)[0])
    np.random.shuffle(indx)
    # split 60-20-20%
    split1 = int(np.shape(Z.T)[0]*0.6)
    split2 = int(np.shape(Z.T)[0]*0.8)
    X_train = Z.T[indx[:split1]]
    Y_train = Y[indx[:split1]]
    X_test = Z.T[indx[split1:split2]]
    Y_test = Y[indx[split1:split2]]
    X_CV = Z.T[indx[split2:]]
    Y_CV = Y[indx[split2:]]


    # train SVM and loop through all C and gamma combinations
    for j in range(len(C)):
        for k in range(len(gamma)):
            #print '      C and gamma:',C[j],gamma[k]
            SVC_bud = SVC(kernel='rbf', C=C[j], gamma=gamma[k]).fit(X_train, Y_train)            
            train_score[i,j,k] = SVC_bud.score(X_train,Y_train)
            test_score[i,j,k] = SVC_bud.score(X_test,Y_test)
            CV_score[i,j,k] = SVC_bud.score(X_CV,Y_CV)


    # find the C and gamma parameters that give max score.
    # if there are multiple parameter configuration giving max score, the first one of these is used below 
    best_params = np.where(test_score[i,:,:] == np.max(test_score[i,:,:]))
    print '   best C value(s):',C[best_params[0]]
    print '   best gamma value(s):',gamma[best_params[1]]
    print '   max test score:',np.max(test_score[i,:,:])



avg_score = np.mean(test_score[:,:,:],axis=0)
avg_CV_score = np.mean(CV_score[:,:,:],axis=0)
std_CV_score = np.std(CV_score[:,:,:],axis=0)

best_params = np.where(avg_score == np.max(avg_score))
print 'best C value(s):',C[best_params[0]]
print 'best gamma value(s):',gamma[best_params[1]]
print 'max test score:',np.max(avg_score)

print 'CV score:',avg_CV_score[best_params],'+/-',std_CV_score[best_params]
# train the best SVM and save it
SVC_bud = SVC(kernel='rbf', C=C[best_params[0][0]], gamma=gamma[best_params[1][0]]).fit(Z.T, Y)            


# save the results
f = open('SVM_tile_scores_nsim'+str(n_sim)+'.dat','w')
pickle.dump([test_score,train_score,CV_score,SVC_bud,C,gamma],f)
f.close()



    0
   best C value(s): [   100.   1000.  10000.]
   best gamma value(s): [ 0.001  0.001  0.001]
   max test score: 0.770833333333
    1
   best C value(s): [    10.    100.   1000.  10000.]
   best gamma value(s): [ 0.001  0.001  0.001  0.001]
   max test score: 0.7375
    2
   best C value(s): [  1.00000000e+00   1.00000000e+01   1.00000000e+02   1.00000000e+02
   1.00000000e+03   1.00000000e+03   1.00000000e+04   1.00000000e+04]
   best gamma value(s): [ 0.01   0.01   0.001  0.01   0.001  0.01   0.001  0.01 ]
   max test score: 0.745833333333
    3
   best C value(s): [    10.    100.   1000.  10000.]
   best gamma value(s): [ 0.01  0.01  0.01  0.01]
   max test score: 0.7625
    4
   best C value(s): [ 10.]
   best gamma value(s): [ 0.001]
   max test score: 0.783333333333
    5
   best C value(s): [    10.    100.   1000.  10000.]
   best gamma value(s): [ 0.01  0.01  0.01  0.01]
   max test score: 0.7625
    6
   best C value(s): [ 1.]
   best gamma value(s): [ 0.001]
   max te

In [ ]:
# render the image
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams['figure.figsize'] = (9.0, 6.0)
from sklearn.svm import SVC
from skimage.util.shape import view_as_windows
import pickle

f = open('PCA_data_30x30_var099.dat','r')
[Z, U_reduced, k] = pickle.load(f)
f.close()

f = open('SVM_tile_scores_nsim100.dat','r')
[test_score,train_score,CV_score,SVC_bud,C,gamma] = pickle.load(f)
f.close()

windows = view_as_windows(im1,(30,30))

print np.shape(Z)[0]

PCA_features = np.zeros([np.shape(windows)[0],np.shape(windows)[1],np.shape(Z)[0]])

class_img = np.zeros([np.shape(windows)[0],np.shape(windows)[1]])

# collect the PCA features

mean_im = np.mean(im1)
std_im = np.std(im1)
for i in range(np.shape(windows)[0]):
    if i%100 == 0:
        print i
    for j in range(np.shape(windows)[1]):

        tile_standard = (windows[i,j] - mean_im)/std_im
        reshape_tile = tile_standard.reshape(30*30)
        PCA_features[i,j,:] = np.dot(reshape_tile,U_reduced)

# predict the classes:
for i in range(np.shape(windows)[0]):
    if i%100 == 0:
        print i
    class_img[i,:] = SVC_bud.predict(PCA_features[i,:,:])

plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.axis('equal')
plt.xlim([0,np.shape(im1)[0]])
plt.ylim([0,np.shape(im1)[1]])
plt.imshow(im1,cmap='Greys_r')
#plt.scatter(cr1,cc1,color='r')

plt.subplot(1, 2, 2)
plt.xlim([0,np.shape(hog_image)[0]])
plt.ylim([0,np.shape(hog_image)[1]])
plt.imshow(class_img)
plt.savefig('imgs/class_tiles_size'+str(30)+'.png',dpi=150)
plt.close()


99
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276